In [66]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import kagglehub

path = kagglehub.dataset_download("mosapabdelghany/medical-insurance-cost-dataset")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\annad\.cache\kagglehub\datasets\mosapabdelghany\medical-insurance-cost-dataset\versions\1


In [67]:
data = pd.read_csv(f"{path}/insurance.csv")
print("Информация о данных:")
data.info()
print("Размер данных до подготовки:", data.shape)

Информация о данных:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB
Размер данных до подготовки: (1338, 7)


In [68]:
print("Пропуски в данных:")
data.isnull().sum()

Пропуски в данных:


age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

In [69]:
num_columns = ["age", "bmi", "children", "charges"]
categorial_columns = list(set(data.columns) - set(num_columns))
print("численные:", num_columns)
print("категориальные:", categorial_columns)

численные: ['age', 'bmi', 'children', 'charges']
категориальные: ['sex', 'region', 'smoker']


In [70]:
mean_vals = data[num_columns].mean()
std_vals = data[num_columns].std()

In [71]:
mask = (np.abs(data[num_columns] - mean_vals) > 3 * std_vals).any(axis=1)
data_clean = data[~mask].copy()
print(f"Удалено строк из-за выбросов: {data.shape[0] - data_clean.shape[0]}")
print("Размер данных после удаления выбросов:", data_clean.shape)


Удалено строк из-за выбросов: 29
Размер данных после удаления выбросов: (1309, 7)


In [72]:
categorial_columns = list(set(data_clean.columns) - set(num_columns))
# Drop_first=True для избежания ловушки фиктивных переменных ((Справа: One-Hot Encoding (фиктивные переменные), 
# где каждая категория преобразуется в новый бинарный признак. Чтобы избежать ловушки фиктивных переменных (Multicollinearity), 
# обычно удаляют один из столбцов, созданных для категориального признака.))
final_data = pd.get_dummies(data_clean, columns=categorial_columns, drop_first=True)

print("Признаки после One-Hot Encoding:")
final_data.head()

Признаки после One-Hot Encoding:


,age,bmi,children,charges,sex_male,region_northwest,region_southeast,region_southwest,smoker_yes
0,19,27.900,0,16884.92400,False,False,False,True,True
1,18,33.770,1,1725.55230,True,False,True,False,False
2,28,33.000,3,4449.46200,True,False,True,False,False
3,33,22.705,0,21984.47061,True,True,False,False,False
4,32,28.880,0,3866.85520,True,True,False,False,False


In [73]:
print("Матрица корреляции:")
final_data.corr()

Матрица корреляции:


,age,bmi,children,charges,sex_male,region_northwest,region_southeast,region_southwest,smoker_yes
age,1.000000,0.118178,0.059100,0.305263,-0.019253,-0.003449,-0.014346,0.013766,-0.029090
bmi,0.118178,1.000000,0.029916,0.191453,0.042498,-0.134342,0.261014,0.002552,-0.005288
children,0.059100,0.029916,1.000000,0.100438,0.014056,0.044948,-0.027381,0.001275,0.028844
charges,0.305263,0.191453,0.100438,1.000000,0.059455,-0.045847,0.074829,-0.042922,0.785129
sex_male,-0.019253,0.042498,0.014056,0.059455,1.000000,-0.006602,0.013987,-0.004798,0.079058
region_northwest,-0.003449,-0.134342,0.044948,-0.045847,-0.006602,1.000000,-0.346405,-0.321537,-0.040557
region_southeast,-0.014346,0.261014,-0.027381,0.074829,0.013987,-0.346405,1.000000,-0.341411,0.071776
region_southwest,0.013766,0.002552,0.001275,-0.042922,-0.004798,-0.321537,-0.341411,1.000000,-0.039005
smoker_yes,-0.029090,-0.005288,0.028844,0.785129,0.079058,-0.040557,0.071776,-0.039005,1.000000


Вывод по Подготовке данных:
Пропусков в исходном наборе данных не обнаружено, что упрощает работу. Было удалено 29 строк (из 1338) по правилу трех стандартных отклонений $(\mu \pm 3\sigma)$ для всех численных признаков. Это потенциально может улучшить производительность линейной модели, так как выбросы особенно сильно влияют на метод наименьших квадратов. Самая сильная корреляция наблюдается между признаком smoker_yes и целевой переменной charges ($\approx 0.785$). Это ожидаемо: курение - это сильный фактор риска, увеличивающий стоимость страховки. Также значимая корреляция с age ($\approx 0.305$). Так как корреляции линейные, многомерная линейная регрессия должна хорошо справиться с этой задачей.

In [74]:
# X признаки и Y целевая переменная
Y = final_data["charges"].to_numpy(dtype=np.double)
X = final_data.drop("charges", axis=1).to_numpy(dtype=np.double)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [75]:
from sklearn.base import RegressorMixin

class AnaliticRegression(RegressorMixin):
    def __init__(self, regularization=0.0):
        self.regularization = regularization 

    def fit(self, X, Y):
        X_design = np.column_stack([np.ones(X.shape[0]), X]) 
        
        m = X_design.shape[1]
        E = np.eye(m)
        E[0, 0] = 0
        
        # Расчет W = (X^T * X + lambda * E)^-1 * X^T * Y
        A = X_design.T @ X_design + self.regularization * E
        B = X_design.T @ Y
        W_full = np.linalg.solve(A, B)
        
        self.b = W_full[0]
        self.w = W_full[1:]

    def predict(self, X):
        return X @ self.w + self.b

anal_ols = AnaliticRegression(regularization=0.0)
anal_ols.fit(X_train_scaled, Y_train)

predicted_anal_ols = anal_ols.predict(X_test_scaled)
mae_anal_ols = mean_absolute_error(Y_test, predicted_anal_ols)
rmse_anal_ols = np.sqrt(mean_squared_error(Y_test, predicted_anal_ols))

print(f"Веса (w): {anal_ols.w}")
print(f"Смещение (b): {anal_ols.b:,.2f}")
print(f"MAE (Аналитически OLS): {mae_anal_ols:,.2f}")
print(f"RMSE (Аналитически OLS): {rmse_anal_ols:,.2f}")

Веса (w): [3649.33512945 1924.02859237  629.28757949  -30.77749084 -214.45651175
 -470.59944911 -511.63739358 9208.30258541]
Смещение (b): 13,021.30
MAE (Аналитически OLS): 3,969.03
RMSE (Аналитически OLS): 5,517.62


In [93]:
class MyLinearRegression(RegressorMixin):
    def __init__(self, step=0.01, max_steps=5000, alpha=0.9, regularization=0.0):
        self.learning_rate = step
        self.max_steps = max_steps
        self.alpha = alpha # Коэффициент моментума
        self.regularization = regularization
        self.w = None
        self.b = 0.0

    def fit(self, X, y):
        n, m = X.shape
        self.w = np.zeros(m) 
        self.b = y.mean()

        prev_grad_w = np.zeros(m) 
        prev_grad_b = 0.0
        
        for i in range(self.max_steps):
            predictions = X @ self.w + self.b
            error = predictions - y # (Xw + b - Y)
            
            grad_w = (2 / n) * (X.T @ error) + (2 * self.regularization * self.w)
            
            grad_b = (2 / n) * np.sum(error)
            
            mgrad_w = prev_grad_w * self.alpha + grad_w * (1 - self.alpha)
            mgrad_b = prev_grad_b * self.alpha + grad_b * (1 - self.alpha)
            
            prev_grad_w = mgrad_w
            prev_grad_b = mgrad_b
            
            self.w -= self.learning_rate * mgrad_w
            self.b -= self.learning_rate * mgrad_b

        return self

    def predict(self, X):
        return X @ self.w + self.b

gd_ols = MyLinearRegression(step=0.01, max_steps=5000, alpha=0.0)
gd_ols.fit(X_train_scaled, Y_train)

predicted_gd_ols = gd_ols.predict(X_test_scaled)
mae_gd_ols = mean_absolute_error(Y_test, predicted_gd_ols)
rmse_gd_ols = np.sqrt(mean_squared_error(Y_test, predicted_gd_ols))

print(f"Веса (w): {gd_ols.w}")
print(f"Смещение (b): {gd_ols.b:,.2f}")
print(f"MAE (GD OLS): {mae_gd_ols:,.2f}")
print(f"RMSE (GD OLS): {rmse_gd_ols:,.2f}")

Веса (w): [3649.33512945 1924.02859237  629.28757949  -30.77749084 -214.45651175
 -470.59944911 -511.63739358 9208.30258542]
Смещение (b): 13,021.30
MAE (GD OLS): 3,969.03
RMSE (GD OLS): 5,517.62


И аналитическое решение (Нормальное уравнение), и численное решение (Градиентный Спуск) для OLS-регрессии дали идентичные и оптимальные результаты на тестовой выборке (RMSE $\approx 5,517.62$). Это подтверждает, что обе наши реализации верны и градиентный спуск достиг минимума функции потерь. Таким образом, для данной нерегуляризованной задачи линейной регрессии аналитический метод является точным, а численный метод показал высокую сходимость.

In [77]:
lambda_val = 1.0

anal_ridge = AnaliticRegression(regularization=lambda_val)
anal_ridge.fit(X_train_scaled, Y_train)

predicted_anal_ridge = anal_ridge.predict(X_test_scaled)
mae_anal_ridge = mean_absolute_error(Y_test, predicted_anal_ridge)
rmse_anal_ridge = np.sqrt(mean_squared_error(Y_test, predicted_anal_ridge))

print(f"Веса (w) с Ridge (лямбда={lambda_val}): {anal_ridge.w}")
print(f"Смещение (b): {anal_ridge.b:,.2f}")
print(f"MAE (Аналитически Ridge): {mae_anal_ridge:,.2f}")
print(f"RMSE (Аналитически Ridge): {rmse_anal_ridge:,.2f}")

Веса (w) с Ridge (лямбда=1.0): [3646.01818704 1921.9736294   629.18997252  -29.98517725 -213.8831276
 -468.4099675  -510.65218567 9199.25965008]
Смещение (b): 13,021.30
MAE (Аналитически Ridge): 3,970.26
RMSE (Аналитически Ridge): 5,517.67


In [94]:
gd_ridge = MyLinearRegression(step=0.01, max_steps=5000, alpha=0.0, regularization=lambda_val)
gd_ridge.fit(X_train_scaled, Y_train)

predicted_gd_ridge = gd_ridge.predict(X_test_scaled)
mae_gd_ridge = mean_absolute_error(Y_test, predicted_gd_ridge)
rmse_gd_ridge = np.sqrt(mean_squared_error(Y_test, predicted_gd_ridge))

print(f"Веса (w) с Ridge (лямбда={lambda_val}): {gd_ridge.w}")
print(f"Смещение (b): {gd_ridge.b:,.2f}")
print(f"MAE (GD Ridge): {mae_gd_ridge:,.2f}")
print(f"RMSE (GD Ridge): {rmse_gd_ridge:,.2f}")

Веса (w) с Ridge (лямбда=1.0): [1856.16025284  973.2346693   449.32294566  181.32594719 -138.77827921
  116.49325625 -252.40593468 4559.10810579]
Смещение (b): 13,021.30
MAE (GD Ridge): 5,591.94
RMSE (GD Ridge): 7,581.45


Аналитическое решение для Ridge-регрессии показало отличный результат (RMSE $\approx 5,517.67$), который очень близок к OLS. Это говорит о том, что для этого набора данных L2-регуляризация при $\lambda=1.0$ не дает существенного прироста в точности. Градиентный Спуск для Ridge с теми же параметрами дал аномально плохой результат (RMSE $\approx 7,581.45$). Это указывает на проблемы сходимости численного метода

In [83]:
class MeanBaselineModel(RegressorMixin):
    def fit(self, X, Y):
        self.mean_value = np.mean(Y)
    
    def predict(self, X):
        return self.mean_value * np.ones(X.shape[0])

models = [
    ('Константная (Среднее)', MeanBaselineModel()),
    ('OLS (Аналитически)', anal_ols),
    ('OLS (Градиентный Спуск, alpha=0)', gd_ols),
    (f'Ridge (Аналитически, лямба={lambda_val})', anal_ridge),
    (f'Ridge (Градиентный Спуск, лямбда={lambda_val}, alpha=0)', gd_ridge)
]

results = []

for name, model in models:
    if not hasattr(model, 'w') or not hasattr(model, 'mean_value'):
        model.fit(X_train_scaled, Y_train)
    
    if isinstance(model, MeanBaselineModel):
        predictions = model.predict(X_test)
    else:
        predictions = model.predict(X_test_scaled)
        
    mse = mean_squared_error(Y_test, predictions)
    rmse = np.sqrt(mse)
    
    results.append({'Модель': name, 'MSE': mse, 'RMSE': rmse})

results_df = pd.DataFrame(results).sort_values(by='MSE').reset_index(drop=True)

print("Таблица результатов на тестовой выборке (по MSE):")
results_df['MSE'] = results_df['MSE'].apply(lambda x: f"{x:,.2f}")
results_df['RMSE'] = results_df['RMSE'].apply(lambda x: f"{x:,.2f}")
print(results_df.to_markdown(index=False))

Таблица результатов на тестовой выборке (по MSE):
| Модель                                         | MSE            | RMSE      |
|:-----------------------------------------------|:---------------|:----------|
| OLS (Аналитически)                             | 30,444,091.53  | 5,517.62  |
| OLS (Градиентный Спуск, alpha=0)               | 30,444,091.53  | 5,517.62  |
| Ridge (Аналитически, лямба=1.0)                | 30,444,628.51  | 5,517.67  |
| Ridge (Градиентный Спуск, лямбда=1.0, alpha=0) | 57,478,399.03  | 7,581.45  |
| Константная (Среднее)                          | 137,709,080.74 | 11,734.95 |


Константная модель (прогноз средним значением) имеет самый высокий MSE ($\approx 138$ млн), что ожидаемо. Линейные модели (OLS и Ridge) показали себя значительно лучше, снизив MSE более чем в 4 раза (до $\approx 30.4$ млн). Это подтверждает сильную линейную связь между признаками, прежде всего, с курением. Аналитическое решение OLS и GD OLS являются лучшими моделями по точности (MSE $\approx 30,444,091.53$), что указывает на отсутствие необходимости в регуляризации для данного набора данных.Ridge (Аналитически) также показала высокую точность (MSE $\approx 30,444,628.51$). Однако Ridge (Градиентный Спуск) продемонстрировала аномально низкое качество (MSE $\approx 57,478,399.03$). Аналитическое решение OLS является лучшим выбором, поскольку оно дает точное и надежное решение. Провал GD при $\lambda=1.0$ для Ridge, в то время как GD OLS был точен, указывает на то, что, возможно, численному методу в условиях регуляризации требуется более тонкая настройка параметров.